In [1]:
# notebook settings
%load_ext autoreload
%autoreload 2

# external imports
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt

# internal imports
from pympc.dynamics.discrete_time_systems import MixedLogicalDynamicalSystem
from pympc.control.hybrid_benchmark.controller_mld import HybridModelPredictiveController
from pympc.plot import plot_input_sequence, plot_state_trajectory, plot_output_trajectory

In [2]:
# system numeric parameters
mc = 1.
mp = 1.
l = 1.
d = 1.
k = 100.
g = 10.
h = .05

# state bounds
x_max = np.array([d, np.pi/6., 7., 5.])
x_min = - x_max

# input domain
u_max = np.array([20.])
u_min = - u_max

In [3]:
# symbolic states
x = sp.Matrix(sp.symbols('q t qd td'))

# symbolic continuous inputs
u = sp.Matrix([sp.symbols('u')])

# symbolic binary inputs
ub = sp.zeros(0,1)

# symbolic forces (left, right)
f = sp.Matrix(sp.symbols('fl fr'))

# symbolic auxiliary binaries (left, right)
b = sp.Matrix(sp.symbols('bl br'))

# dynamics
x_next = sp.Matrix([
    x[0] + h*x[2],
    x[1] + h*x[3],
    x[2] + h*((g*mp)/mc*x[1] + u[0]/mc),
    x[3] + h*(g*(mc+mp)/(l*mc)*x[1] + u[0]/(l*mc) - f[0]/(l*mp) + f[1]/(l*mp))
])

In [4]:
# state bounds
state_upper_bound = x - x_max.reshape(x_max.size, 1)
state_lower_bound = x_min.reshape(x_min.size, 1) - x

# input bounds
input_upper_bound = u - u_max.reshape(u_max.size, 1)
input_lower_bound = u_min.reshape(u_min.size, 1) - u

# contact left wall (big-M method)
# positive penetration: - d - x[0] + l*x[1]
left_force = sp.Matrix([
    - f[0],
    k*(- d - x[0] + l*x[1]) - f[0],
    f[0] - b[0]*k*(- d - x_min[0] + l*x_max[1]),
    f[0] - k*(- d - x[0] + l*x[1]) + (1. - b[0])*k*(- d - x_max[0] + l*x_min[1])
])

# contact right wall (big-M method)
# positive penetration: x[0] - l*x[1] - d
right_force = sp.Matrix([
    - f[1],
    k*(x[0] - l*x[1] - d) - f[1],
    f[1] - b[1]*k*(x_max[0] - l*x_min[1] - d),
    f[1] - k*(x[0] - l*x[1] - d) + (1. - b[1])*k*(x_min[0] - l*x_max[1] - d)
])

# gather constraints
constraints = sp.Matrix([
    state_upper_bound,
    state_lower_bound,
    input_upper_bound,
    input_lower_bound,
    left_force,
    right_force
])

In [5]:
# construct MLD system
MLD = MixedLogicalDynamicalSystem.from_symbolic(x, u, ub, f, b, x_next, constraints)

In [6]:
# controller parameters
N = 50
Q = 10.*np.eye(MLD.nx)*h/2.
R = .1*np.eye(MLD.nuc)*h/2.
P = Q

# hybrid controller
controller = HybridModelPredictiveController(MLD, N, P, Q, R)

In [7]:
x0 = np.array([0., 0., 2.5, 0.])

In [8]:
sol, leaves = controller.feedforward(x0, tree_file_name='bb_mld')

|     Updates    |     Time (s)   |    Nodes (#)   |   Lower bound  |   Upper bound  |
 ---------------- ---------------- ---------------- ---------------- ----------------
  Root node        0.052            1                4.508            inf             
                   5.064            112              38.373           inf             
                   10.107           219              61.315           inf             
  New incumbent    15.084           324              63.516           63.516          
  Solution found   15.133           325              63.516           63.516          

Explored 325 nodes in 15.133 seconds: optimal solution found with objective 63.516.


In [9]:
len(leaves)

163

In [18]:
np.random.seed(1)
e_max = 0.005
e = e_max * np.random.randn(MLD.nx)

In [19]:
uc_0 = sol['primal']['uc'][0]
ub_0 = sol['primal']['ub'][0]
sc_0 = sol['primal']['sc'][0]
sb_0 = sol['primal']['sb'][0]
warm_start = controller.generate_warm_start(leaves, x0, uc_0, ub_0, sc_0, sb_0, e)

In [20]:
len(warm_start)

161

In [21]:
x1 = sol['primal']['x'][1] + e
controller.feedforward(x1, tree_file_name='bb_mld', warm_start=warm_start)

Warm start loaded with lower bound 57.607.
|     Updates    |     Time (s)   |    Nodes (#)   |   Lower bound  |   Upper bound  |
 ---------------- ---------------- ---------------- ---------------- ----------------
  Root node        0.078            1                59.226           inf             
  New incumbent    4.754            84               63.333           63.333          
  Solution found   4.818            85               63.333           63.333          

Explored 85 nodes in 4.818 seconds: optimal solution found with objective 63.333.


({'dual': {'alpha': [array([-118.521405  ,  337.73335077, -111.10328473,  118.81103203]),
    array([-118.45484413,  284.35726657, -103.43186295,  105.09048628]),
    array([-118.30084929,  237.95452384,  -95.35424978,   94.10320411]),
    array([-118.03872591,  196.94166032,  -87.59083595,   84.87714018]),
    array([-117.68352866,  160.13676665,  -80.31075264,   77.06017963]),
    array([-117.25853606,  126.47072127,  -73.50196414,   70.52643099]),
    array([-116.78625038,   94.97372243,  -67.16208816,   65.17693582]),
    array([-116.28893652,   64.74821207,  -61.29391845,   60.94132204]),
    array([-49.46420933,  36.05651174, -58.82070799,  58.12099661]),
    array([-262.86808054,  238.94667024,  -45.84276066,   45.00017772]),
    array([-392.10650599,  355.95493743,  -26.62950414,   25.94603052]),
    array([-420.36045277,  381.89222794,   -6.21972217,    5.7452141 ]),
    array([-362.36837433,  330.09563962,   11.09511536,  -11.51007335]),
    array([-246.32815872,  226.1385137

In [ ]:
plot_input_sequence(sol['uc'], h, (u_min, u_max))
plt.show()
plot_state_trajectory(sol['x'], h, (x_min, x_max))
plt.show()

In [ ]:
plot_input_sequence(sol['sc'], h)
plt.show()

In [ ]:
plot_input_sequence(sol['sb'], h)
plt.show()

In [ ]:
# output (horizontal position of the tip of the pole)
C = np.array([[1., -l, 0., 0.]])
y_min = np.array([-d])
y_max = np.array([d])
plot_output_trajectory(C, sol['x'], h, (y_min, y_max))
plt.show()

In [ ]:
from collections import OrderedDict
ident = OrderedDict([(('s', 0, 0), 0.0), (('s', 0, 1), 0.0)])

In [ ]:
sol = controller.solve_relaxation(x0, ident)[3]

In [ ]:
sol['primal']['sb_0']

In [ ]:
min(np.concatenate([sol[3]['dual']['lb_s_'+str(t)] for t in range(N)]))

In [ ]:
# print np.max(np.concatenate([v for v in sol['dual'].values()]))

In [ ]:
[sol['dual']['lb_s_%d'%t] for t in range(N)]
[np.concatenate((sol['dual']['lb_u_%d'%t], sol['dual']['lb_s_%d'%t])) for t in range(N)]

In [ ]:
import time
import meshcat
import meshcat.geometry as g
import meshcat.transformations as tf

In [ ]:
vis = meshcat.Visualizer()

In [ ]:
# cart-pole
cart_pole = vis["cart_pole"]

# cart
cart = cart_pole["cart"]
cart.set_object(g.Box([.3*l, .3*l, .3*l]), g.MeshLambertMaterial(color=0xff2222))

# pivot
pivot = cart["pivot"]
pivot.set_transform(tf.rotation_matrix(np.pi/2, [1, 0., 0.]))

# pole
pole = pivot["pole"]
pole.set_object(g.Box([.05*l, l, .05*l]), g.MeshLambertMaterial(color=0x2222ff))
pole.set_transform(tf.translation_matrix([0., .5, 0.]))

# left wall
left_wall = vis["left_wall"]
left_wall.set_object(g.Box([l, .05*l, l]), g.MeshLambertMaterial(color=0x22ff22))
left_wall.set_transform(tf.translation_matrix([0., -d, l]))

# right wall
right_wall = vis["right_wall"]
right_wall.set_object(g.Box([l, .05*l, l]), g.MeshLambertMaterial(color=0x22ff22))
right_wall.set_transform(tf.translation_matrix([0., d, l]))

In [ ]:
# visualization function
def visualize_cart_pole(x):
    cart.set_transform(tf.translation_matrix([0, x[0], 0]))
    pivot.set_transform(tf.rotation_matrix(x[1] + np.pi/2, [1, 0, 0]))
vis.jupyter_cell()

In [ ]:
# animation
for t in range(N):
    visualize_cart_pole(sol['primal']['x'][t])
    time.sleep(h)